## RAG Implentation locally

In [1]:
from langchain_community.llms import Ollama 

llm = Ollama(model = "zephyr")

In [2]:
llm.invoke("Tell me a short joke")

"Why don't scientists trust atoms? Because they make up everything."

### Load data for RAG

In [3]:
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader(
    web_path="https://blog.langchain.dev/langgraph/"
)

docs = loader.load()

### Index the data


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True 
)

In [5]:
all_splits = text_splitter.split_documents(docs)


In [7]:
from langchain_community import embeddings 

embedding = embeddings.ollama.OllamaEmbeddings(
    model="nomic-embed-text"
)

In [16]:
# turn this into embedding .. Let's Use OpenAI Model
from langchain.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

In [17]:
from langchain_community.vectorstores import Chroma 

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding
)

In [19]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [20]:
retriever.get_relevant_documents("What is LangGraph?")


[Document(page_content="TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we've implemented already. We will then highlight a few of the common modifications to these runtimes we've heard requests for, and examples of implementing those. We will finish with a preview of what we will be releasing next.MotivationOne of the big val

### Create a chain to contextualize


In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

In [22]:
contextualize_q_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [23]:
from langchain_core.output_parsers import StrOutputParser

contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [24]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_chain.invoke(
    {
        "chat_history":[
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large?",
    }
)

'"In what context are we referring to \'large\' when discussing a large language model?"'

#### Create a chain for chat history


In [25]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

In [26]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [27]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [28]:
def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]

In [29]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    RunnablePassthrough.assign(
        context = contextualized_question | retriever | format_docs
    )
    | qa_prompt 
    | llm
)

Inference


In [30]:
chat_history = []

question = "What is LangGraph?"
ai_msg = rag_chain.invoke(
    {
        "question": question,
        "chat_history": chat_history
    }
)

In [31]:
ai_msg 


"LangGraph is a library built on top of LangChain that enables creating cyclical graphs for agent runtimes. It provides an interface to represent the graph and update its state through nodes that return operations to attributes in the form of a key-value store. The state can be overridden or updated by adding new values. This adds value to LangChain's ability to create custom chains, especially in situations where cycles are needed for agentic behavior. LangGraph helps construct state machines with more control over how tools are called and different prompts based on the state of the agent."

In [32]:
chat_history.extend(
    [
        HumanMessage(content=question), ai_msg
    ]
)

In [33]:
second_question = "What is it used for?"

rag_chain.invoke(
    {
        "question": second_question,
        "chat_history": chat_history
    }
)

"LangGraph, built on top of LangChain, enables the creation of cyclical graphs for agent runtimes in LLM applications, which is often needed for agentic behavior. It provides a StateGraph class that represents the graph and updates its state through nodes that return operations to attributes in the form of a key-value store. The state can be overridden or updated by adding new values. LangGraph helps construct state machines with more control over how tools are called and different prompts based on the state of the agent. It adds value to LangChain's ability to create custom chains, especially in situations where cycles are needed for agentic behavior."

### Worked Example 2 

In [38]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [40]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [42]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.9/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


Started /Users/akashvarun/.cache/weaviate-embedded: process ID 80294


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-04-13T23:57:38-04:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-04-13T23:57:38-04:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-04-13T23:57:38-04:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-04-13T23:57:38-04:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-04-13T23:57:38-04:00"}
adding route HEAD /v1/objects/{id} "objects.head"
operation: spec.Operation{Vendor

In [43]:
retriever = vectorstore.as_retriever()

In [44]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "What did the president say about Justice Breyer"
rag_chain.invoke(query)

/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.9/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  response = response.dict()
/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.9/site-packages/pydantic/main.py:962: PydanticDeprecat

"The president honored Justice Breyer for his service and dedication to the country. He mentioned nominating Judge Ketanji Brown Jackson to continue Justice Breyer's legacy of excellence. The president did not provide specific comments about Justice Breyer's work or character."

{"action":"read_disk_use","level":"warning","msg":"disk usage currently at 89.52%, threshold set to 80.00%","path":"/Users/akashvarun/.local/share/weaviate","time":"2024-04-13T23:58:09-04:00"}
